In [2]:
import time
from io import BytesIO
from datetime import datetime

import pandas as pd
import win32clipboard
from PIL import Image
from selenium import webdriver
from selenium.webdriver import Keys
from selenium.webdriver.common.by import By

In [3]:
options = webdriver.ChromeOptions()
options.add_argument("disable-infobars")  # Deshabilitar infobars
options.add_argument("--disable-notifications")  # Deshabilitar notifications

options.add_experimental_option("useAutomationExtension", False)
options.add_experimental_option("excludeSwitches", ["enable-automation"])

In [5]:
cService = webdriver.ChromeService(executable_path='./drivers/chromedriver.exe')
browser = webdriver.Chrome(service=cService)

In [6]:
browser.get("https://web.whatsapp.com/")

In [7]:
def paste_content(driver, el, content):
    driver.execute_script(
        f'''
        const text = `{content}`;
        const dataTransfer = new DataTransfer();
        dataTransfer.setData('text', text);
        const event = new ClipboardEvent('paste', {{clipboardData: dataTransfer, bubbles: true}});
        arguments[0].dispatchEvent(event)''',
        el)

In [8]:
def send_to_clipboard(clip_type, image_data):
    win32clipboard.OpenClipboard()
    win32clipboard.EmptyClipboard()
    win32clipboard.SetClipboardData(clip_type, image_data)
    win32clipboard.CloseClipboard()

In [9]:
def calculate_diff_dates(fecha_str):
    # Convertir la cadena de fecha a un objeto datetime
    fecha_dada = datetime.strptime(fecha_str, "%d/%m/%Y")
    # Obtener la fecha actual
    fecha_actual = datetime.now()
    # Calcular la diferencia en días
    diferencia_dias = (fecha_actual - fecha_dada).days
    
    # Calcular los meses y días
    if diferencia_dias > 0:
        meses = diferencia_dias // 30
        dias = diferencia_dias % 30
    else:
        meses = 0
        dias = diferencia_dias
    
    return meses, dias

In [10]:
def generate_message_date(fecha_input):
    meses, dias = calculate_diff_dates(fecha_input)
    if meses == 1:
        atraso = f"{meses} mes"
    elif meses > 1:
        atraso = f"{meses} meses"
    else:
        atraso = f"{dias} días"
    return atraso

In [11]:
## Load image to Bytes
filepath = "image_promo.jpeg"
image = Image.open(filepath)

output = BytesIO()
image.convert("RGB").save(output, "BMP")
image_data = output.getvalue()[14:]
output.close()

In [12]:
message_promotion = """
Hola {0}. Se le informa que presenta un atraso de {1} en el pago de cuenta. Su CODIGO de cliente es: **{2}**
"""

In [13]:
## Bucle
# Lee el archivo CSV en un DataFrame
df = pd.read_csv("cobranzas.csv")

# Itera sobre cada fila del DataFrame
for index, row in df.iterrows():
    try:
        print(str(row['TELEFONO']))
        time.sleep(0.2)
        browser.find_element(By.XPATH,
                             "//div[@id='app']/div/div[2]/div[3]/header/header/div/span/div/span/div/div/span").click()
        time.sleep(0.2)
        browser.find_element(By.XPATH,
                             "//*[@id='app']/div/div[2]/div[2]/div[1]/span/div/span/div/div[1]/div[2]/div[2]/div/div").send_keys(
            str(row['TELEFONO']))
        time.sleep(1)
        
        if len(browser.find_elements(By.XPATH, "//*[@id='app']/div/div[2]/div[2]/div[1]/span/div/span/div/div[2]/div[2]")) == 0:
            browser.find_element(By.XPATH, "").click()
            continue
            
        browser.find_element(By.XPATH, "//*[@id='app']/div/div[2]/div[2]/div[1]/span/div/span/div/div[2]/div[2]").click()
        time.sleep(0.2)
        ## Send image
        send_to_clipboard(win32clipboard.CF_DIB, image_data)
        browser.find_element(
            By.XPATH,
            "//*[@id='main']/footer/div[1]/div/span/div/div[2]/div[1]/div/div[1]/p").send_keys(Keys.CONTROL + "v")
        time.sleep(0.3)
        ## Send Text Promotion
        send_text = browser.find_element(By.XPATH,
                                         "//*[@id='app']/div/div[2]/div[2]/div[2]/span/div/div/div/div[2]/div/div[1]/div[3]/div/div/div[2]/div[1]/div[1]/p")
        paste_content(browser, send_text, message_promotion.format(row['NOMBRE'], generate_message_date(row['FECHA_VENCIDO']), row['CODIGO']))
        ## Send Message
        browser.find_element(By.XPATH,
                             "//*[@id='app']/div/div[2]/div[2]/div[2]/span/div/div/div/div[2]/div/div[2]/div[2]/div/div").click()
        
        ## Actualizado de csv
        df.at[index, 'ESTADO_ENVIADO'] = 1
    except Exception as e:
        print(f"Error processing row {index}: {e}")
        
# Guardar el DataFrame actualizado en un nuevo archivo CSV
df.to_csv("cobranzas.csv", index=False)

51 955 427 797
Error processing row 0: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[@id='app']/div/div[2]/div[3]/header/header/div/span/div/span/div/div/span"}
  (Session info: chrome=131.0.6778.86); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF6A8906CB5+28821]
	(No symbol) [0x00007FF6A8873840]
	(No symbol) [0x00007FF6A871578A]
	(No symbol) [0x00007FF6A87691BE]
	(No symbol) [0x00007FF6A87694AC]
	(No symbol) [0x00007FF6A87B2647]
	(No symbol) [0x00007FF6A878F33F]
	(No symbol) [0x00007FF6A87AF412]
	(No symbol) [0x00007FF6A878F0A3]
	(No symbol) [0x00007FF6A875A778]
	(No symbol) [0x00007FF6A875B8E1]
	GetHandleVerifier [0x00007FF6A8C3FCAD+3408013]
	GetHandleVerifier [0x00007FF6A8C5741F+3504127]
	GetHandleVerifier [0x00007FF6A8C4B5FD+3455453]
	GetHandleVerifier [0x00007FF6A89CBDBB+835995]
	(No symbol) [0x00007FF